In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

response = requests.get('https://ru.citaty.net/avtory/artemii-andreevich-lebedev/')
print('Visited URL: {}'.format(response.url))
print(response.status_code)

Visited URL: https://ru.citaty.net/avtory/artemii-andreevich-lebedev/
200


In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
# Находим теги с сообщениями
data_containers = soup.find_all('p', class_='blockquote-text')
dat = []
for dc in data_containers:
    # Очищаем сообщения от мусора
    quote = dc.text.replace('\xa0', ' ')
    match = re.search(r'„(.*?)“', quote)
    # Формируем список сообщений
    dat.append(match.group(1))

quotes = pd.DataFrame(data=dat, columns=['Message'])
display(quotes)

,Message
0,Как мотивировать себя что-то делать?— Да никак...
1,"Церковь должна показывать, что она богаче меня..."
2,"Дизайнер живёт своей профессией, делая из хаос..."
3,"Человек становится взрослым и самостоятельным,..."
4,"В какой-то момент кто-то создает дизайн, насто..."
5,"Молодой дизайнер, впервые столкнувшийся с тем,..."
6,"Люди обожают, когда пульт дистанционного управ..."
7,Есть два типа людей: одни праздники организовы...
8,Компьютер позволяет быстро-быстро создавать мн...
9,Ваша визитка — говно.


In [14]:
data_before = 0
messages_list = []
count_pages = 10
for _ in range(count_pages):
    result = requests.get(f"https://t.me/s/temalebedev?before={data_before}")
    soup = BeautifulSoup(result.content.decode('utf8'), 'html.parser')
    # Находим блоки с сообщениями
    data_containers = soup.find_all('div', class_='tgme_widget_message_text js-message_text')
    dat = []
    # Создаем список стоп слов
    stop_words = ["Реклама", "нативнаяинтеграция", "erid", "http"]
    # Исключаем посты со стоп словами (Данные посты могут быть написаны другим человеком)
    for dc in data_containers:
        if not any(word in dc.text for word in stop_words):  
            dat.append(dc.text)
    # Находим тег <a>
    a_tag = soup.find('a', class_='tme_messages_more js-messages_more')
    # Получаем значение атрибута data-before для перехода на следующую страницу
    data_before = a_tag['data-before'] if a_tag else None
    # Формируем список сообщений
    messages_list += dat

tg_messages = pd.DataFrame(data=messages_list, columns=['Message'])
display(tg_messages)


,Message
0,Заю прощаю!
1,Ответственное потреблениеВот в моем детстве вс...
2,"Божоле нувоПиздец, самое хуевое пойло на свете..."
3,"РавенствоКаждый раз, когда кто-то требует раве..."
4,Левостороннее движениеЯ водил во многих страна...
...,...
134,Зубная щеткаПро зубную щетку нужно знать самое...
135,"Сила подписиМне нравится история про Рубенса, ..."
136,Карточки безопасности в самолетахУ меня есть м...
137,"Зай, посмотрел пропагандистский русофобский ук..."


In [16]:
combined_df = pd.concat([tg_messages, quotes], ignore_index=True)
display(combined_df)

,Message
0,Заю прощаю!
1,Ответственное потреблениеВот в моем детстве вс...
2,"Божоле нувоПиздец, самое хуевое пойло на свете..."
3,"РавенствоКаждый раз, когда кто-то требует раве..."
4,Левостороннее движениеЯ водил во многих страна...
...,...
161,Размер пикселя — это как размер ангела: точных...
162,Дизайн спасет мир.
163,Критика — это хорошо. Ничто так не вдохновляет...
164,Маркетинга не существует.
